In [2]:
import json
import os
import sys
from typing import Dict, List
import torch

import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoModelForCausalLM



import trlx
from trlx.data.configs import (
    ModelConfig,
    OptimizerConfig,
    SchedulerConfig,
    TokenizerConfig,
    TrainConfig,
    TRLConfig,
)
from trlx.models.modeling_ppo import PPOConfig

from trlx.models.modeling_ppo import (
    AutoModelForCausalLMWithHydraValueHead,
    AutoModelForCausalLMWithValueHead,
    AutoModelForSeq2SeqLMWithHydraValueHead,
)






In [3]:
    def reward_se( prompts: List[str], outputs: List[str], **kwargs) -> List[float]:

        rewards = []
        for q, a in zip(prompts, outputs):
            feedback_prompt = f'Is the answer to the question correct? The question is: {q}. The answer is: {a}'
            feedback = se_generator(feedback_prompt)[0]['generated_text']  # Assuming 'model' is your trained T5 model
            feedback = feedback.lower().strip()
            print(feedback)
            reward = 0.0 
            if 'yes' in feedback:
                reward = 1.0 
                
            elif 'no' in feedback:
                reward = -1.0

            rewards.append(reward)
        return rewards

# Task

In [6]:
task='navigate'

# load models

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
    
    # Load the model
model_se_0 = T5ForConditionalGeneration.from_pretrained("/root/autodl-tmp/flan-t5-large")

    # Load the tokenizer
tokenizer_se = T5Tokenizer.from_pretrained("/root/autodl-tmp/flan-t5-large")

    # Create the pipeline
se_generator= pipeline("text2text-generation", model=model_se_0, tokenizer=tokenizer_se,
                        do_sample= True,
                        top_k= 50,
                        top_p= 0.95,
                        max_length=100,
                        eos_token_id= -1,
                        temperature= 1.0,
                          device=0 if int(os.environ.get("LOCAL_RANK", 0)) == 0 else -1,)

In [19]:
    # Load the model
model_se_1 = T5ForConditionalGeneration.from_pretrained(f"/root/autodl-tmp/msc_ml/t5_large_checkpoints/{task}/checkpoint_2000")



    # Create the pipeline
se_generator_1 = pipeline("text2text-generation", model=model_se_1, tokenizer=tokenizer_se,
                        do_sample= True,
                        top_k= 50,
                        top_p= 0.95,
                        max_length=100,
                        eos_token_id= -1,
                        temperature= 1.0,
                        device=0 if int(os.environ.get("LOCAL_RANK", 0)) == 0 else -1,)


Some weights of the model checkpoint at /root/autodl-tmp/msc_ml/t5_large_checkpoints/web_of_lies/checkpoint_2000 were not used when initializing T5ForConditionalGeneration: ['v_head.0.weight', 'v_head.2.bias', 'v_head.0.bias', 'v_head.2.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
ds = load_dataset("json", data_files=f"/root/autodl-tmp/BIG-Bench-Hard/bbh/{task}.json",field="examples")['train']


"""
ds_split=ds.train_test_split(test_size=0.2)
prompt_all=ds['input']
prompt_all_new= [prompt.replace('\n', ' ') for prompt in prompt_all]

prompt_train=ds_split['train']['input']
prompt_test=ds_split['test']['input']

answer_all=ds['target']
answer_test=ds_split['test']['target']
answer_train=ds_split['train']['target']
prompt_test_new= ['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_test]
prompt_train_new= ['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_train]
"""




    
prompt_all=ds['input']
# fix the train test split
train_test_split_id=round(len(prompt_all)*0.8)
prompt_train=prompt_all[:train_test_split_id]
prompt_test=prompt_all[train_test_split_id:]  

# ds_split=ds.train_test_split(test_size=0.2)
#prompt_train=ds_split['train']['input']
#prompt_test=ds_split['test']['input']
    
    


    
prompt_all= ['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_all]
prompt_test= ['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_test]
prompt_train= ['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_train]  

answer_all=ds['target']
answer_train=answer_all[:train_test_split_id]
answer_test=answer_all[train_test_split_id:]
    

    



def accuracy(prompts: List[str], outputs: List[str]) -> Dict[str, List[float]]:
    match=[]
        
    for i,prompt in enumerate(prompts):

        index = prompt_all_new.index(prompt)
        if outputs[i].lower().strip()==answer_all[index].lower().strip():
            is_correct=1.0
        else:
            is_correct=0.0
                
        match.append(is_correct)

    return sum(match)/len(match)

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-e5badb54ef3cd267/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
import tqdm

test_epochs=4

outputs_0=[]
outputs_1=[]

     
batch_size = 10  

for epoch in range(test_epochs):
    for i in tqdm.tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size

        # Slice the data to create a batch
        batch_questions = prompt_test[i:i+batch_size]
        batch_answers = answer_train[i:i+batch_size]


        batch_generations_0 = se_generator(batch_questions,batch_size=batch_size)
        outputs_0.extend([gen['generated_text'] for gen in batch_generations_0])

        # Assuming se_generator_1 can handle batches
        batch_generations_1 = se_generator_1(batch_questions,batch_size=batch_size)
        outputs_1.extend([gen['generated_text'] for gen in batch_generations_1])
        
        
"""
for epoch in range(test_epochs):
    for i in tqdm.tqdm(range(len(prompt_test))):


        question=prompt_test[i]
        answer=answer_test[i]
        #print('££££££££££££££££££££££££££',question)
        generation_0=se_generator(question)[0]['generated_text']
        outputs_0.append(generation_0)
        #print('$$$$$$$$$$$$$',generation)
        #print('============',answer)
        #print(reward_fn(prompts=[question], outputs=[generation]))



        generation_1=se_generator_1(question)[0]['generated_text']
        outputs_1.append(generation_1)
        #print('$$$$$$$$$$$$$',generation)
        #print('============',answer)
        #print(reward_fn(prompts=[question], outputs=[generation]))
"""

100%|██████████| 5/5 [00:38<00:00,  7.71s/it]


"\nfor epoch in range(test_epochs):\n    for i in tqdm.tqdm(range(len(prompt_test))):\n\n\n        question=prompt_test[i]\n        answer=answer_test[i]\n        #print('££££££££££££££££££££££££££',question)\n        generation_0=se_generator(question)[0]['generated_text']\n        outputs_0.append(generation_0)\n        #print('$$$$$$$$$$$$$',generation)\n        #print('============',answer)\n        #print(reward_fn(prompts=[question], outputs=[generation]))\n\n\n\n        generation_1=se_generator_1(question)[0]['generated_text']\n        outputs_1.append(generation_1)\n        #print('$$$$$$$$$$$$$',generation)\n        #print('============',answer)\n        #print(reward_fn(prompts=[question], outputs=[generation]))\n"

In [22]:
def accuracy(prompts: List[str], outputs: List[str]) -> Dict[str, List[float]]:
    match=[]
        
    for i,prompt in enumerate(prompts):
        generation=''
        if 'yes' in outputs[i].lower().strip():
            generation='yes'
        elif 'no' in outputs[i].lower().strip():
            generation='no'
            

        index = prompt_all.index(prompt)
        if generation==answer_all[index].lower().strip():
            is_correct=1.0
        else:
            is_correct=0.0
                
        match.append(is_correct)

    return sum(match)/len(match)

In [23]:
accuracy_0=accuracy(prompt_test*4,outputs_0)
print(accuracy_0)
accuracy_1=accuracy(prompt_test*4,outputs_1)
print(accuracy_1)

0.5
0.42


In [1]:
import time 
time.ctime()

'Fri Jul 28 23:53:52 2023'

In [10]:
prompt_all

['[If you follow these instructions, do you return to the starting point? Always face forward. Take 1 step backward. Take 9 steps left. Take 2 steps backward. Take 6 steps forward. Take 4 steps forward. Take 4 steps backward. Take 3 steps right. Options: - Yes - No] Let’ s think step by step.',
 '[If you follow these instructions, do you return to the starting point? Always face forward. Take 10 steps left. Take 10 steps forward. Take 7 steps forward. Take 2 steps forward. Options: - Yes - No] Let’ s think step by step.',
 '[If you follow these instructions, do you return to the starting point? Take 1 step. Take 10 steps. Take 4 steps. Take 1 step. Take 10 steps. Turn right. Turn around. Options: - Yes - No] Let’ s think step by step.',
 '[If you follow these instructions, do you return to the starting point? Always face forward. Take 1 step right. Take 3 steps left. Take 2 steps right. Options: - Yes - No] Let’ s think step by step.',
 '[If you follow these instructions, do you return